# 2020-01-28

In this notebook I will test the pickling process (open a file), then move on to trying to do sentiment analysis on the comments:

In [1]:
import pickle
with open('comms_spa.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

In [2]:
import pickle
import pandas as pd
import random
import numpy as np
import pandas as pd
import re

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg

from tqdm.notebook import tqdm
from pprint import pprint

In [23]:
mynewlist[10]

i love every single human in this video!!

In [20]:
type(mynewlist[10])

spacy.tokens.doc.Doc

### So the pickling process works, now onto linking the comments found with the commenter / video / channel details

In [38]:
insight_dir = 'C:\\Users\\Ronald Maj\\Documents\\GitHub\\InsightDataProject\\'

# Load vids data
vids_df = pd.read_csv(insight_dir+'data\\processed\\all_vids_en_df.csv', index_col=0)
# Load comments data
comms_df = pd.read_csv(insight_dir+'data\\processed\\all_comments_en_df.csv', index_col=0)

In [39]:
comms_df = comms_df.dropna(subset=['textDisplay'])

In [46]:
comms_df = comms_df.drop(['Unnamed: 0.1'], axis=1)

In [47]:
# For our new sample, just get a random assortment of 10,000 comments from teh data frame:
docs_samp_df = comms_df.sample(10000)

In [50]:
docs_samp_df.head(5)

,CommID,authorChannelId,authorChannelUrl,authorDisplayName,authorProfileImageUrl,canRate,likeCount,moderationStatus,parentId,publishedAt,textDisplay,textOriginal,updatedAt,videoId,viewerRating
856135,UgxVFSUP6-XbCL7OMHN4AaABAg,{'value': 'UCxXXtHrgiORgmYPVZ_wnv2g'},http://www.youtube.com/channel/UCxXXtHrgiORgmY...,Cailyn Richer,https://yt3.ggpht.com/a/AGF-l7-ZWJopAKvifvgm_E...,True,1,NaN,0,2020-01-25T17:26:30.000Z,“You wouldn’t DIY your own medical treatment” ...,“You wouldn’t DIY your own medical treatment” ...,2020-01-25T17:26:30.000Z,p9JpaK4Wr-E,none
359633,UgxPOubJHk09vKZ0k294AaABAg,{'value': 'UC8HTipSO3eCCstuZJ0UyvIQ'},http://www.youtube.com/channel/UC8HTipSO3eCCst...,Sarah Margaret Miles,https://yt3.ggpht.com/a/AGF-l7-jjowAwqPd4ASyV1...,True,1,NaN,0,2019-11-13T18:07:44.000Z,Daniel: “this is war” \nLmaoooo,Daniel: “this is war” \nLmaoooo,2019-11-13T18:07:44.000Z,6WAkZcRnKEQ,none
267013,Ugw-LpUbvxO_Lwv--fJ4AaABAg,{'value': 'UCzL_W5zNCZrTCw_8M9EhMDg'},http://www.youtube.com/channel/UCzL_W5zNCZrTCw...,Bram jr Duijvenvoorde,https://yt3.ggpht.com/a/AGF-l7_xj0q7gXb4n4UE9I...,True,0,NaN,0,2019-11-18T20:53:08.000Z,What an inspiring video🙌 I’d love to see more ...,What an inspiring video🙌 I’d love to see more ...,2019-11-18T20:53:08.000Z,0OCEo2n7vLw,none
348722,UgzDc_jAsy3gpUgs5Lp4AaABAg,{'value': 'UCJ7W3Fo-tdywhSmjtJrNAdg'},http://www.youtube.com/channel/UCJ7W3Fo-tdywhS...,FL Gwynn,https://yt3.ggpht.com/a/AGF-l7_F9q5nfktRAylSEN...,True,1,NaN,0,2019-09-15T16:24:55.000Z,Awww the puppy are so cute but when they are f...,Awww the puppy are so cute but when they are f...,2019-09-15T16:24:55.000Z,ITlVaxwciuE,none
559933,UgwWD--2014evA3MPEF4AaABAg,{'value': 'UCn1PmZJfe7oWvY3y_uPatTA'},http://www.youtube.com/channel/UCn1PmZJfe7oWvY...,cass naee,https://yt3.ggpht.com/a/AGF-l7_L0GxVf4PH0DjeLC...,True,2,NaN,0,2020-01-08T08:48:40.000Z,Never seen someone so obsessed with being shor...,Never seen someone so obsessed with being shor...,2020-01-08T08:48:40.000Z,zqIzIkJbvq8,none


In [48]:
comm_docs = docs_samp_df['textDisplay']

In [51]:
def clean_text_spa(text):
    # lower case:
    text = text.lower()
    # replace new line and return with space
    text = text.replace("\n", " ").replace("\r"," ")
    # replace punctuation marks with space
    punc_list = '#/<>@[\]{}|^~'
    t=str.maketrans(dict.fromkeys(punc_list, ""))
    #text = text.translate(t)
    # replace single quote with empty character
    #t = str.maketrans(dict.fromkeys("'`",""))
    #text = text.translate(t)
    return text

comm_docs = [clean_text_spa(doc) for doc in comm_docs]


# Cleaning emojis
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

comm_docs = [deEmojify(doc) for doc in comm_docs]

<>:7: DeprecationWarning: invalid escape sequence \]
<>:7: DeprecationWarning: invalid escape sequence \]
<>:7: DeprecationWarning: invalid escape sequence \]
<ipython-input-51-f06d53f7a3fa>:7: DeprecationWarning: invalid escape sequence \]
  punc_list = '#/<>@[\]{}|^~'


In [52]:
comm_docs[0:5]

['you wouldnt diy your own medical treatment  anti-vaxxers: ',
 'daniel: this is war  lmaoooo',
 'what an inspiring video id love to see more of this',
 'awww the puppy are so cute but when they are fighting they need lark to make them stop plus u will need to give them wet food instead of dry',
 'never seen someone so obsessed with being short like them ']

### Load spaCy language model

In [54]:
nlp2 = spacy.load("en_core_web_lg")

C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\msgpack_numpy.py:184: DeprecationWarnin

In [56]:
spa_docs= []
for doc in tqdm(docs_samp):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp2(doc)
    spa_docs.append(pr)

KeyboardInterrupt: 